In [ ]:
from gurobipy.gurobipy import *
import numpy as np
import time
import math
import pandas as pd

In [ ]:
print(gurobi.version())

In [ ]:
Countries = ['Germany', 'France', 'UK', 'Italy', 'Spain', 'Poland', 'Romania', 'Netherlands', 'Belgium', 'Greece', 'Czech Republic',
            'Portugal', 'Hungary', 'Sweden', 'Austria','Bulgaria','Denmark','Finland', 'Slovakia', 'Ireland', 'Croatia', 'Lithuania',
            'Slovenia','Latvia','Estonia','Cyprus','Luxembourg','Malta']
Populations = [80780000, 65856609, 64308261, 60782668, 46507760, 38495659, 19942642, 16829289, 11203992, 10992589, 10512419,
              10427301, 9879000, 9644864, 8507786, 7245677, 5627235, 5451270, 5415949, 4604029, 4246700, 2943472, 2061085,
              2001468, 1315819, 858000, 549680, 425384]
D = {Countries[i]: Populations[i] for i in range(len(Countries))}

In [ ]:
Original = ['Germany', 'France', 'Italy', 'Netherlands', 'Belgium', 'Luxembourg']
EU73 = Original + ['Denmark', 'Ireland']
EU86 = EU73 + ['Greece', 'Portugal', 'Spain']
EU95 = EU86 + ['Austria', 'Finland', 'Sweden']
EU04 = EU95 + ['Cyprus', 'Czech Republic', 'Estonia', 'Hungary', 'Latvia', 'Lithuania', 'Malta', 'Poland', 'Slovakia', 'Slovenia']

In [ ]:
losers = {1 : [1,4,7,13,14], 
          2 : [2,3,6,13],
          3 : [1,4,5,28],
          4 : [1,2,10,11,14],
          5 : [1,3,8,11,15],
          6 : [1,4,8,9,12],
          7 : [1,2,7,17,18],
          8 : [1,5,6,16,18],
          9 : [2,4,6,15,21],
          10 : [1,3,9,10,12],
          11 : [3,4,5,20,22],
          12 : [2,3,7,8,9],
          13 : [1,3,6,26],
          14 : [2,3,5,19],
          15 : [16,17,18,19,20,21,22,23,24,25,26,27,28]
         }
Y_KoberWeltge = []
for i in range(15):
    y = [1 for i in range(28)]
    for k in range(28):
        if k+1 in losers[i+1]:
            y[k] = 0
    Y_KoberWeltge.append(tuple(y))

In [ ]:
def binary(n, votes, code = list(), k = 0, ):
    """Gives all tuples of length n containing zeroes and ones"""
    all_votes = votes
    if len(code) == n:#Do we have a tuple of length n
        all_votes.append(tuple(code))
    else:
        code.append(0)
        all_votes = binary(n, all_votes, code, k+1)#recursive call
        code.pop()
        code.append(1)
        all_votes = binary(n, all_votes, code, k+1)#recursive call
        code.pop()
    return all_votes

In [ ]:
def generate_X_Y(all_votes, c_2):
    "Given all possible votes, it seperates them into yes votes (X) and no votes (Y)"
    X=[]
    Y=[]
    for v in all_votes:
        if (sum(v)>=len(c_2)-3) or ((sum(v)>=math.ceil(len(c_2)*0.55)) and (sum([v[i]*c_2[i] for i in vote])>=(sum(c_2)*0.65))):
            X.append(v)
        else:
            Y.append(v)
    return X, Y

In [ ]:
def make_Y_prime(i, c_2, rand):
    "Randomly generate a set Y' of size i for given population and random seed"
    Y = []
    k=0
    np.random.seed(rand)#init random seed
    s = len(c_2)
    while k< i:
        y = np.random.randint(2, size=s)#generate random y
        if sum(y)< math.ceil(s*0.55) and sum(y)< (s-3) and tuple(y) not in Y:#Check if y violates rules 1 and 2
            Y.append(tuple(y))
            k += 1
        elif (sum(y)<(s-3)) and (sum([y[i]*c_2[i] for i in vote])<(sum(c_2)*0.65)) and (tuple(y) not in Y):#check if y violates rule 3
            Y.append(tuple(y))
            k += 1
    return Y

In [ ]:
def init_I_prime(Y):
    "Initialize I' by defining each set in it to contain one of the vectors in Y"
    I_prime = []
    for y in Y:
        I_prime.append((y,))
    return I_prime

In [ ]:
def init_basic_model(I_prime, Y):
    "Basic model to find the dimension"
    model = Model('EU')
    model.Params.LogFile = 'EU_model'
    model.Params.LogToConsole = 0
    Z = dict()
    for i in I_prime:
        Z[i] = model.addVar(
            obj = 1.0, name='', lb=0, ub=GRB.INFINITY
        )#define variables
    model.modelSense = GRB.MINIMIZE
    constraints = model.addConstrs(
        (quicksum(Z[i] for i in I_prime if y in i)>= 1 for y in Y),
        name = 'Constraint'
    )#define constraints
    model.optimize()
    return model, Z, constraints

In [ ]:
def init_X(k, c_2, rand):
    "Randomly initialize set X given a size k and a random seed"
    i = 0
    X = []
    np.random.seed(rand)
    s = len(c_2)
    while i< k:
        x = np.random.choice([0,1], size=s, p = [1./3, 2./3])#randomly generate x with bias
        if sum(x)>= s-3 and tuple(x) not in X:#check condition 3
            X.append(tuple(x))
            i = i+1
        elif (sum(x)>=math.ceil(s*0.55)) and (sum([x[i]*c_2[i] for i in vote])>=(sum(c_2)*0.65)) and (tuple(x) not in x):#check condition 1 and 2
            X.append(tuple(x))
            i = i+1
    return X

In [ ]:
def get_alpha(Y, model):
    "Extract the shadow price from the model"
    alpha = alpha = model.getAttr(GRB.Attr.Pi)
    alp = {y: 1 for y in Y}
    for i in range(len(Y)):
        alp[Y[i]] = alpha[i]
    return alp

In [ ]:
def init_shadow_model(Y, X, alp, vote, epsilon = 0.001):
    "Second linear program to check if dimension found is optimal"
    model_1 = Model('EU_1')
    model_1.Params.LogFile = 'EU_model_1'
    model_1.Params.LogToConsole = 0
    U = model_1.addVars(Y
        , name='U', vtype=GRB.BINARY 
    )
    a = model_1.addVars(vote, name='a', lb = -1, ub = 1)
    beta = model_1.addVar(name='b', lb = -1*len(vote), ub = len(vote))
    model_1.setObjective(
        quicksum(alp[y]*U[y] for y in Y),
        sense=GRB.MAXIMIZE
    )
    constraint_1 = dict()
    for x in X:
        constraint_1[x] = model_1.addConstr(
            (quicksum(a[i]*x[i] for i in vote)<= beta),
            name = ''
        )
    constraint_2 = dict()
    for y in Y:
        constraint_2[y] = model_1.addConstr(
            quicksum(a[i]*y[i] for i in vote) >= beta + epsilon - (2*len(vote) + epsilon)*(1 - U[y]),
            #(U[y] == 1) >> (quicksum(a[i]*y[i] for i in vote)>= beta + epsilon),
            name = ''
        )
    model_1.optimize()
    model_1.write("anotherdebug.lp")
    a_list = [a[i].x for i in vote]
    return model_1, U, a, beta, constraint_1, constraint_2, a_list

In [ ]:
def first_x(a_list, vote, c_2):
    "Check if Linear inequality holds for all x for which 1 and 2 hold"
    model_2 = Model('EU_2')
    model_2.Params.LogFile = 'EU_model_2'
    model_2.Params.LogToConsole = 0
    x_1 = model_2.addVars(vote
        , name='x', vtype=GRB.BINARY 
    )
    model_2.setObjective(
        quicksum(a_list[i]*x_1[i] for i in vote),
        sense=GRB.MAXIMIZE
    )
    constraint_3 = model_2.addConstr(
        ((quicksum(x_1[i] for i in vote)>= math.ceil(len(vote)*0.55))),
        name = 'Constraint_1'
    )
    constraint_4 = model_2.addConstr(
        (quicksum(x_1[i]*c_2[i] for i in vote)>=sum(c_2)*0.65)
    )
    model_2.optimize()
    return model_2, x_1, constraint_3, constraint_4

In [ ]:
def second_x(a_list, vote, c_2):
    "Check if linear inequality holds for all x for which condition 3 holds"
    model_3 = Model('EU_3')
    model_3.Params.LogFile = 'EU_model_3'
    model_3.Params.LogToConsole = 0
    x_2 = model_3.addVars(vote
        , name='x', vtype=GRB.BINARY 
    )
    model_3.setObjective(
        quicksum(a_list[i]*x_2[i] for i in vote),
        sense=GRB.MAXIMIZE
    )    
    constraint_5 = model_3.addConstr(
        ((quicksum(x_2[i] for i in vote) >= len(vote)-3)),
        name = 'Constraint_1'
    )
    model_3.optimize()
    return model_3, x_2, constraint_5

In [ ]:
def check_X(model_1, U, a_list, be, vote, X, generate, c_2):
    "Check if linear inequality holds for all x and rerun linear program until it does"
    init_time = time.time()
    truth = True
    X_new = []
    i = 0
    while truth:
        model_2, x_1, contraint_3, constraint_4 = first_x(a_list, vote, c_2)
        model_3, x_2, constraint_5 = second_x(a_list, vote, c_2)
        obj_2 = model_2.getObjective().getValue()
        obj_3 = model_3.getObjective().getValue()
            if obj_2 == max(obj_2,obj_3):
                new_x = tuple([i.x for i in x_1.values()])
            else:
                new_x = tuple([i.x for i in x_2.values()])
        else:
            truth = False
            break
        X.append(new_x)
        if generate == True:
            X = Gen_x(list(new_x), X, vote, c_2)
        model_1.addConstr(
            (quicksum(a[i]*new_x[i] for i in vote)<= beta),
            name = ''
        )
        model_1.optimize()
        be = beta.x
        a_list = [a[i].x for i in vote]
        X_new.append(new_x)
        i +=1
    return model_1, U, a_list, be, X_new, X, i

In [ ]:
def Gen_x(x, X, vote, c_2):
    "Generate additional x"
    for i in vote:
        if x[len(vote)-i-1]==1:
            x[len(vote)-i-1]=0
            if (sum(x)< math.ceil(len(vote)*0.55) or sum([x[i]*c_2[i] for i in vote])<sum(c_2)*0.65) and sum(x)<len(vote)-3:
                x[len(vote)-i-1]=1
                if x not in X:
                    X.append(tuple(x))
    return X

## Code to run the method once

In [ ]:
n=10
Example = [Countries[i] for i in range(n)]
c_2 = [D[i] for i in Example]
vote = [i for i in range(len(c_2))]
nr_Y = 10
nr_X = 10
rand = 0
generate = False
Y_complete = True
X_complete = True
KoberWeltge = False
times = []
start_time = time.time()
new_ys = []
obj = 5
if not X_complete and not Y_complete:
    Y = make_Y_prime(nr_Y, c_2, rand)
    X = init_X(nr_X, c_2, rand)
elif X_complete and not Y_complete:
    Y = make_Y_prime(nr_Y, c_2, rand)
    all_votes = binary(len(c_2), []) 
    X,not_Y = generate_X_Y(all_votes, c_2)
else:
    all_votes = binary(len(c_2), []) 
    X,Y = generate_X_Y(all_votes, c_2)
if KoberWeltge:
    Y = Y_KoberWeltge
I_prime = init_I_prime(Y)
model, Z, constraints = init_basic_model(I_prime, Y)
if generate:
    for i in range(len(X)):
        x = X[i]
        X = Gen_x(list(x), X, vote, c_2)
alp = get_alpha(Y, model)
while obj>1:
    model_1, U, a, beta, constraint_1, constraint_2, a_list = init_shadow_model(Y, X, alp, vote)
    shadow_time = time.time() - start_time
    times.append(shadow_time)
    be = beta.x
    if not X_complete:
        model_1, U, a_list, be, X_new, X, iterations = check_X(model_1, U, a_list, be, vote, X, generate, c_2)
    check_time = time.time() - start_time
    times.append(check_time)
    obj = model_1.getObjective().getValue()
    if obj<=1:
        break
    I_new = []
    for y in Y:
        if U[y].x == 1:
            I_new.append(y)
    c = []
    new_ys.append(I_new)
    for y in Y:
        if y in I_new:
            c.append(1)
        else:
            c.append(0)
    constrs = model.getConstrs()
    new_column = Column(c, constrs)
    Z[tuple(I_new)] = model.addVar(obj = 1, lb=0, ub=GRB.INFINITY,name = '', column = new_column)
    pre_opt_time = time.time() - start_time
    times.append(pre_opt_time)
    model.optimize()
    post_opt_time = time.time() - start_time
    times.append(post_opt_time)
    alp = get_alpha(Y, model)
final_time = time.time() - start_time
times.append(final_time)

In [ ]:
final_time

In [ ]:
times

In [ ]:
print(f'Optimal objective value: {model.objVal}\n')
for i in Z.values():
    if i.x > 0:
        print(f'{i.varName} = {i.x}')
model.write("debug.lp")

## Code to iterate over different values

In [ ]:
experiment_1 = pd.DataFrame(columns=['Size_Y', 'Seed', 'Final_time','Iprime_iterations' , 'x_iterations', 'EU_Size', 'Dimension'])
y_sizes = [10,]
seeds = [0,]
EU_sizes = [12,14,16,18,20,22,24,26]
EU_config = [Original, EU73, EU86, EU95, EU04]
for size in EU_sizes:
    for nr_Y in y_sizes:
        for rand in seeds:
            Example = [Countries[i] for i in range(size)]
            c_2 = [D[i] for i in Example]
            vote = [i for i in range(len(c_2))]
            nr_X = 10
            generate = False
            Y_complete = False
            X_complete = False
            times = []
            start_time = time.time()
            new_ys = []
            obj = 5
            if not X_complete and not Y_complete:
                Y = make_Y_prime(nr_Y, c_2, rand)
                X = init_X(nr_X, c_2, rand)
            elif X_complete and not Y_complete:
                Y = make_Y_prime(nr_Y, c_2, rand)
                all_votes = binary(len(c_2), []) 
                X,not_Y = generate_X_Y(all_votes, c_2)
            else:
                all_votes = binary(len(c_2), []) 
                X,Y = generate_X_Y(all_votes, c_2)
            I_prime = init_I_prime(Y)
            model, Z, constraints = init_basic_model(I_prime, Y)
            if generate:
                for i in range(len(X)):
                    x = X[i]
                    X = Gen_x(list(x), X, vote, c_2)
            alp = get_alpha(Y, model)
            count = 0
            while obj>1:
                model_1, U, a, beta, constraint_1, constraint_2, a_list = init_shadow_model(Y, X, alp, vote)
                shadow_time = time.time() - start_time
                times.append(shadow_time)
                be = beta.x
                #print(be)
                if not X_complete:
                    model_1, U, a_list, be, X_new, X, iterations = check_X(model_1, U, a_list, be, vote, X, generate, c_2)
                check_time = time.time() - start_time
                times.append(check_time)
                obj = model_1.getObjective().getValue()
                if obj<=1:
                    break
                I_new = []
                for y in Y:
                    if U[y].x == 1:
                        I_new.append(y)
                c = []
                new_ys.append(I_new)
                for y in Y:
                    if y in I_new:
                        c.append(1)
                    else:
                        c.append(0)
                constrs = model.getConstrs()
                new_column = Column(c, constrs)
                Z[tuple(I_new)] = model.addVar(obj = 1, lb=0, ub=GRB.INFINITY,name = '', column = new_column)
                pre_opt_time = time.time() - start_time
                times.append(pre_opt_time)
                model.optimize()
                post_opt_time = time.time() - start_time
                times.append(post_opt_time)
                alp = get_alpha(Y, model)
                count +=1
            final_time = time.time() - start_time
            times.append(final_time)
            dim = 0
            for i in Z.values():
                if i.x > 0:
                    dim += i.x
            new_row = pd.Series([int(nr_Y), int(rand), round(final_time,2),count , int(iterations), size, round(dim,2)], index = experiment_1.columns)
            experiment_1 = experiment_1.append(new_row, ignore_index=True)

In [ ]:
experiment_1['Seed'] = experiment_1['Seed'].astype('int')
experiment_1['Size_Y'] = experiment_1['Size_Y'].astype('int')
experiment_1['x_iterations'] = experiment_1['x_iterations'].astype('int')
experiment_1['Iprime_iterations'] = experiment_1['Iprime_iterations'].astype('int')
experiment_1['EU_Size'] = experiment_1['EU_Size'].astype('int')
experiment_1

In [ ]:
experiment_1.to_csv('experiment3.csv')